In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 1. SREDJIVANJE PODATAKA ZA NEURONSKU MREZU

### 1.1 ANALIZA SKUPA PODATAKA

In [3]:
data = pd.read_csv("maxbet_bez_nedostajucih_vrednosti.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222284 entries, 0 to 222283
Data columns (total 85 columns):
 #   Column                                                         Non-Null Count   Dtype  
---  ------                                                         --------------   -----  
 0   korisnik                                                       222284 non-null  int64  
 1   week in year                                                   222284 non-null  int64  
 2   number of tickets                                              222284 non-null  int64  
 3   verified                                                       222284 non-null  float64
 4   week deposit in                                                222284 non-null  float64
 5   week deposit out                                               222284 non-null  float64
 6   week deposit diff                                              222284 non-null  float64
 7   week deposit diff %                            

In [5]:
data.head()

,korisnik,week in year,number of tickets,verified,week deposit in,week deposit out,week deposit diff,week deposit diff %,frequency,total week payin,total week win,total week diff,average number of combinations,cashout win,cashback win,median_odds,diff between payin and win time,median risk value on tickets,average payin on ticket,median payin on ticket,number of distinct payin on ticket value,number of different ip adress,payin on live,payin on prematch,margin on live,margin on prematch,average payin on live,average payin on prematch,number of authorization ticket,number of approved ticket,problematic_ticket_cluster,stedev odds on ticket,average hit odd,average diff between match kickoff time and ticket payin time,average diff between gray and real odds,average win odd value,average loss odd value,favorite sport,2nd favorite sport,favorite sport payin,2nd favorite sport payin,favorite sport diff,2nd favorite sport diff,% fs in total sport diff,% 2nd fs in total sport diff,% fs in total sport payin,% 2nd fs in total sport payin,favorite league,2nd favorite league,favorite league payin,2nd favorite league payin,favorite league diff,2nd favorite league diff,% fs in total league diff,% 2nd fs in total league diff,% fs in total league payin,% 2nd fs in total league payin,favorite bet,2nd favorite bet,favorite bet payin,2nd favorite bet payin,favorite bet diff,2nd favorite bet diff,% fs in total bet diff,% 2nd fs in total bet diff,favorite no. of matches,2nd favorite no. of matches,favorite no. of matches payin,2nd favorite no. of matches payin,favorite no. of matches diff,2nd favorite no. of matches diff,favourite hour,favourite hour payin,week no. of leagues,week no. of sports,week no. of prematch mathces,week no. of live mathces,week no. of favourite mathces,average week league risk value,week live payin,week live diff,week live margin %,week prematch payin,week prematch diff,week prematch margin %
0,2786,10,21,0.0,3200.0,0.0,3200.0,1.0,10,10130.0,6930.0,3200.0,1.0,0.0,0.0,6.933810,0.0,-36.204381,482.38,482.380952,12,6,700.0,9430.0,3.00,-0.89,33.33,449.05,0,0,A,4.223371,2.918667,0.0,-1.524861,1.254861,1.258194,Fudbal,Specijal košarka,8629.0,200.0,240.0,99.0,0.97,0.02,0.97,0.02,CYPRUS 2,UEFA Europa League,2500.0,661.0,0.0,-147.0,0.28,0.07,0.28,0.07,Ukupno golova main,Tim 2+,3591.0,2508.0,211.0,384.0,0.40,0.28,1.0,4.0,3329.0,2748.0,2886.0,-738.0,14.0,3700.0,2256,35,3081,22362,2051,0.592416,1.087537e+09,1.053392e+08,0.096860,4.935514e+08,4.102456e+07,0.083121
1,2786,11,13,0.0,2640.0,0.0,2640.0,1.0,8,3668.0,1028.0,2640.0,1.0,0.0,0.0,6.036923,0.0,-46.209538,282.15,282.153846,6,5,450.0,3218.0,-0.35,8.80,34.62,247.54,0,0,A,6.142672,3.201667,0.0,-1.500727,1.246727,1.254000,Fudbal,Nema,5493.0,0.0,1402.0,0.0,1.00,0.00,1.00,0.00,GERMANY Bundesliga - Women,ITALY 2,588.0,513.0,588.0,513.0,0.11,0.09,0.11,0.09,Ukupno golova u prvom poluvremenu CG,Tim 2+,1718.0,1213.0,1278.0,663.0,0.31,0.22,1.0,2.0,2668.0,1025.0,615.0,1025.0,13.0,890.0,2389,35,3132,23361,2563,0.599306,1.111456e+09,6.044331e+07,0.054382,4.740170e+08,1.007177e+08,0.212477
2,2786,12,15,0.0,3000.0,0.0,3000.0,1.0,9,6895.0,3895.0,3000.0,1.0,0.0,0.0,7.251333,0.0,-23.943733,459.67,459.666667,7,5,1435.0,5460.0,1.00,4.05,95.67,364.00,0,0,A,9.694937,12.810000,0.0,-1.478235,1.282353,1.193235,Fudbal,Nema,7494.0,0.0,2415.0,0.0,1.00,0.00,1.00,0.00,Euro 2024 - Qualification,WORLD Friendly International,2624.0,1582.0,1249.0,1438.0,0.35,0.21,0.35,0.21,Ukupno golova u prvom poluvremenu CG,"Ukupno Golova 3,5",2060.0,1984.0,-1347.0,1984.0,0.27,0.26,1.0,2.0,4939.0,1255.0,2449.0,-1549.0,17.0,1635.0,2294,34,2642,25002,2106,0.635647,8.388408e+08,5.859334e+07,0.069850,3.365150e+08,3.818964e+07,0.113486
3,2786,13,9,0.0,1000.0,0.0,1000.0,1.0,3,4081.0,3081.0,1000.0,1.0,0.0,0.0,7.330000,0.0,-45.582444,453.44,453.444444,5,2,1984.0,2097.0,1.00,-1.99,220.44,233.00,0,0,A,6.102032,3.813333,0.0,-1.982955,1.342727,1.640227,Fudbal,Specijal košarka,1014.0,500.0,1.0,-133.0,0.58,0.28,0.58,0.28,Poeni igraca EUROLEAGUE,E

### 1.2 IZBACIVANJE KORISNIKA KOJI SU AKTIVNI MANJE OD 2 NEDELJE

In [6]:
dfAktivnosti = pd.DataFrame(data.korisnik.value_counts())

idx = []

for i in dfAktivnosti.index:
    if dfAktivnosti.loc[i, 'korisnik'] >= 2:
        idx.append(i)
    else:
        break

In [7]:
data = data.loc[data.korisnik.isin(idx),:]

In [8]:
data.shape

(218321, 85)

In [9]:
dfAktivnosti.head(5)

,korisnik
163834,41
120098,41
123170,41
122500,41
122014,41


## 1.3 SREDJIVANJE KATEGORICKIH VREDNOSTI

In [10]:
katVar = data.select_dtypes('object').columns.to_list()

In [11]:
type(katVar)

list

In [12]:
katVar

['problematic_ticket_cluster',
 'favorite sport',
 '2nd favorite sport',
 'favorite league',
 '2nd favorite league',
 'favorite bet',
 '2nd favorite bet']

In [13]:
for var in katVar:
    print(var, len(data[var].unique()))

problematic_ticket_cluster 2
favorite sport 36
2nd favorite sport 41
favorite league 3970
2nd favorite league 4499
favorite bet 1672
2nd favorite bet 1843


#### 1.3.1 IZBACIVANJE KATEGORICKIH ATRIBUTA SA PREVISE RAZLICITIH VREDNOSTI

In [14]:
data = data.drop(columns=['favorite league', '2nd favorite league', 'favorite bet', '2nd favorite bet'])

In [15]:
data.problematic_ticket_cluster.value_counts()

A    209206
0      9115
Name: problematic_ticket_cluster, dtype: int64

In [16]:
data.problematic_ticket_cluster.replace(to_replace='A', value='1', inplace=True)

In [17]:
data.head()

,korisnik,week in year,number of tickets,verified,week deposit in,week deposit out,week deposit diff,week deposit diff %,frequency,total week payin,total week win,total week diff,average number of combinations,cashout win,cashback win,median_odds,diff between payin and win time,median risk value on tickets,average payin on ticket,median payin on ticket,number of distinct payin on ticket value,number of different ip adress,payin on live,payin on prematch,margin on live,margin on prematch,average payin on live,average payin on prematch,number of authorization ticket,number of approved ticket,problematic_ticket_cluster,stedev odds on ticket,average hit odd,average diff between match kickoff time and ticket payin time,average diff between gray and real odds,average win odd value,average loss odd value,favorite sport,2nd favorite sport,favorite sport payin,2nd favorite sport payin,favorite sport diff,2nd favorite sport diff,% fs in total sport diff,% 2nd fs in total sport diff,% fs in total sport payin,% 2nd fs in total sport payin,favorite league payin,2nd favorite league payin,favorite league diff,2nd favorite league diff,% fs in total league diff,% 2nd fs in total league diff,% fs in total league payin,% 2nd fs in total league payin,favorite bet payin,2nd favorite bet payin,favorite bet diff,2nd favorite bet diff,% fs in total bet diff,% 2nd fs in total bet diff,favorite no. of matches,2nd favorite no. of matches,favorite no. of matches payin,2nd favorite no. of matches payin,favorite no. of matches diff,2nd favorite no. of matches diff,favourite hour,favourite hour payin,week no. of leagues,week no. of sports,week no. of prematch mathces,week no. of live mathces,week no. of favourite mathces,average week league risk value,week live payin,week live diff,week live margin %,week prematch payin,week prematch diff,week prematch margin %
0,2786,10,21,0.0,3200.0,0.0,3200.0,1.0,10,10130.0,6930.0,3200.0,1.0,0.0,0.0,6.933810,0.0,-36.204381,482.38,482.380952,12,6,700.0,9430.0,3.00,-0.89,33.33,449.05,0,0,1,4.223371,2.918667,0.0,-1.524861,1.254861,1.258194,Fudbal,Specijal košarka,8629.0,200.0,240.0,99.0,0.97,0.02,0.97,0.02,2500.0,661.0,0.0,-147.0,0.28,0.07,0.28,0.07,3591.0,2508.0,211.0,384.0,0.40,0.28,1.0,4.0,3329.0,2748.0,2886.0,-738.0,14.0,3700.0,2256,35,3081,22362,2051,0.592416,1.087537e+09,1.053392e+08,0.096860,4.935514e+08,4.102456e+07,0.083121
1,2786,11,13,0.0,2640.0,0.0,2640.0,1.0,8,3668.0,1028.0,2640.0,1.0,0.0,0.0,6.036923,0.0,-46.209538,282.15,282.153846,6,5,450.0,3218.0,-0.35,8.80,34.62,247.54,0,0,1,6.142672,3.201667,0.0,-1.500727,1.246727,1.254000,Fudbal,Nema,5493.0,0.0,1402.0,0.0,1.00,0.00,1.00,0.00,588.0,513.0,588.0,513.0,0.11,0.09,0.11,0.09,1718.0,1213.0,1278.0,663.0,0.31,0.22,1.0,2.0,2668.0,1025.0,615.0,1025.0,13.0,890.0,2389,35,3132,23361,2563,0.599306,1.111456e+09,6.044331e+07,0.054382,4.740170e+08,1.007177e+08,0.212477
2,2786,12,15,0.0,3000.0,0.0,3000.0,1.0,9,6895.0,3895.0,3000.0,1.0,0.0,0.0,7.251333,0.0,-23.943733,459.67,459.666667,7,5,1435.0,5460.0,1.00,4.05,95.67,364.00,0,0,1,9.694937,12.810000,0.0,-1.478235,1.282353,1.193235,Fudbal,Nema,7494.0,0.0,2415.0,0.0,1.00,0.00,1.00,0.00,2624.0,1582.0,1249.0,1438.0,0.35,0.21,0.35,0.21,2060.0,1984.0,-1347.0,1984.0,0.27,0.26,1.0,2.0,4939.0,1255.0,2449.0,-1549.0,17.0,1635.0,2294,34,2642,25002,2106,0.635647,8.388408e+08,5.859334e+07,0.069850,3.365150e+08,3.818964e+07,0.113486
3,2786,13,9,0.0,1000.0,0.0,1000.0,1.0,3,4081.0,3081.0,1000.0,1.0,0.0,0.0,7.330000,0.0,-45.582444,453.44,453.444444,5,2,1984.0,2097.0,1.00,-1.99,220.44,233.00,0,0,1,6.102032,3.813333,0.0,-1.982955,1.342727,1.640227,Fudbal,Specijal košarka,1014.0,500.0,1.0,-133.0,0.58,0.28,0.58,0.28,500.0,229.0,-133.0,-26.0,0.28,0.13,0.28,0.13,500.0,400.0,-133.0,-199.0,0.28,0.23,3.0,2.0,500.0,400.0,500.0,-257.0,19.0,1984.0,2141,35,1674,20724,1316,0.646716,6.107079e+08,5.127007e+07,0.083952,2.520192e+08,1.323283e+07,0.052507
4,2786,14,10,0.0,1600.0,0.0,1600.0,1.0,6,3948.0,2348.0,1600.0,1.0,0.0,0.0,7.969000,0.0,-28.164200,394.80,39

In [18]:
data.dtypes

korisnik                                                           int64
week in year                                                       int64
number of tickets                                                  int64
verified                                                         float64
week deposit in                                                  float64
week deposit out                                                 float64
week deposit diff                                                float64
week deposit diff %                                              float64
frequency                                                          int64
total week payin                                                 float64
total week win                                                   float64
total week diff                                                  float64
average number of combinations                                   float64
cashout win                                        

#### 1.3.2 PREBACIVANJE problematic_ticket_cluster ATRIBUTA U NUMERICKI TIP PODATAKA

In [19]:
data.problematic_ticket_cluster.value_counts()

1    209206
0      9115
Name: problematic_ticket_cluster, dtype: int64

In [20]:
data.problematic_ticket_cluster = pd.to_numeric(data.problematic_ticket_cluster)

In [21]:
data.problematic_ticket_cluster.dtype

dtype('int64')

#### 1.3.3 ONE HOT ENKODIRANJE PREOSTALIH KATEGORICKIH ATRIBUTA

In [22]:
zaOstalo = data['favorite sport'].value_counts().index.to_list()

In [23]:
zaOstalo

['Fudbal',
 'Košarka',
 'Specijal košarka',
 'Tenis',
 'Stoni Tenis',
 'Fudbal Virtual',
 'Odbojka',
 'Fudbal Specijal',
 'Hokej',
 'Antepost',
 'Rukomet',
 'E-Fudbal',
 'eSport',
 'E-NBA',
 'Vaterpolo',
 'Američki Fudbal',
 'Bejzbol',
 'Special Eurosong',
 'Moto Sport',
 'Strelac na mecu',
 'MMA',
 'Basket 3x3',
 'Odbojka na pesku',
 'Snuker',
 'Fantasy PL',
 'Pikado',
 'Specijal tenis',
 'Biciklizam',
 'Specijal Vaterpolo',
 'Futsal',
 'Specijal Rukomet',
 'Kriket',
 'Specijal Američki Fudbal',
 'Ragbi liga',
 'Boks',
 'Specijal Oskar']

In [24]:
zaOstalo = list(filter(lambda x: x not in ['Fudbal','Košarka','Specijal košarka','Tenis','Stoni Tenis',], zaOstalo))

In [25]:
zaOstalo

['Fudbal Virtual',
 'Odbojka',
 'Fudbal Specijal',
 'Hokej',
 'Antepost',
 'Rukomet',
 'E-Fudbal',
 'eSport',
 'E-NBA',
 'Vaterpolo',
 'Američki Fudbal',
 'Bejzbol',
 'Special Eurosong',
 'Moto Sport',
 'Strelac na mecu',
 'MMA',
 'Basket 3x3',
 'Odbojka na pesku',
 'Snuker',
 'Fantasy PL',
 'Pikado',
 'Specijal tenis',
 'Biciklizam',
 'Specijal Vaterpolo',
 'Futsal',
 'Specijal Rukomet',
 'Kriket',
 'Specijal Američki Fudbal',
 'Ragbi liga',
 'Boks',
 'Specijal Oskar']

In [26]:
data['favorite sport'].replace(to_replace=zaOstalo, value='Ostalo', inplace=True)

In [27]:
data.replace(to_replace='Specijal košarka', value='Košarka', inplace=True)

In [28]:
data['favorite sport'].value_counts()

Fudbal         182057
Košarka         21710
Tenis            8067
Ostalo           4405
Stoni Tenis      2082
Name: favorite sport, dtype: int64

In [29]:
data['2nd favorite sport'].value_counts()

Nema                        123931
Košarka                      47356
Fudbal                       17196
Tenis                        12382
Stoni Tenis                   4304
Hokej                         2692
Odbojka                       1879
Fudbal Virtual                1867
Fudbal Specijal               1705
Rukomet                       1085
Antepost                       635
Strelac na mecu                575
Bejzbol                        484
Vaterpolo                      368
eSport                         330
Američki Fudbal                297
E-Fudbal                       247
Fantasy PL                     203
E-NBA                          175
MMA                            120
Moto Sport                     101
Special Eurosong                93
Pikado                          82
Basket 3x3                      54
Futsal                          39
Odbojka na pesku                37
Snuker                          36
Specijal tenis                  11
Badminton           

In [30]:
zaOstalo2 = data['2nd favorite sport'].value_counts().index.to_list()

In [31]:
zaOstalo2 = list(filter(lambda x: x not in ['Nema','Košarka','Fudbal','Tenis','Stoni Tenis','Hokej'], zaOstalo2))

In [32]:
zaOstalo2

['Odbojka',
 'Fudbal Virtual',
 'Fudbal Specijal',
 'Rukomet',
 'Antepost',
 'Strelac na mecu',
 'Bejzbol',
 'Vaterpolo',
 'eSport',
 'Američki Fudbal',
 'E-Fudbal',
 'Fantasy PL',
 'E-NBA',
 'MMA',
 'Moto Sport',
 'Special Eurosong',
 'Pikado',
 'Basket 3x3',
 'Futsal',
 'Odbojka na pesku',
 'Snuker',
 'Specijal tenis',
 'Badminton',
 'Kriket',
 'Specijal Vaterpolo',
 'Ragbi liga',
 'Specijal Rukomet',
 'Boks',
 'Biciklizam',
 'Specijal Američki Fudbal',
 'Atletika',
 'Ragbi',
 'Specijal Oskar',
 'Zimski Sportovi']

In [33]:
data['2nd favorite sport'].replace(to_replace=zaOstalo2, value='Ostalo', inplace=True)

In [34]:
data['2nd favorite sport'].value_counts()

Nema           123931
Košarka         47356
Fudbal          17196
Tenis           12382
Ostalo          10460
Stoni Tenis      4304
Hokej            2692
Name: 2nd favorite sport, dtype: int64

In [35]:
data = pd.get_dummies(data, columns=['favorite sport', '2nd favorite sport'])

In [36]:
data.head()

,korisnik,week in year,number of tickets,verified,week deposit in,week deposit out,week deposit diff,week deposit diff %,frequency,total week payin,total week win,total week diff,average number of combinations,cashout win,cashback win,median_odds,diff between payin and win time,median risk value on tickets,average payin on ticket,median payin on ticket,number of distinct payin on ticket value,number of different ip adress,payin on live,payin on prematch,margin on live,margin on prematch,average payin on live,average payin on prematch,number of authorization ticket,number of approved ticket,problematic_ticket_cluster,stedev odds on ticket,average hit odd,average diff between match kickoff time and ticket payin time,average diff between gray and real odds,average win odd value,average loss odd value,favorite sport payin,2nd favorite sport payin,favorite sport diff,2nd favorite sport diff,% fs in total sport diff,% 2nd fs in total sport diff,% fs in total sport payin,% 2nd fs in total sport payin,favorite league payin,2nd favorite league payin,favorite league diff,2nd favorite league diff,% fs in total league diff,% 2nd fs in total league diff,% fs in total league payin,% 2nd fs in total league payin,favorite bet payin,2nd favorite bet payin,favorite bet diff,2nd favorite bet diff,% fs in total bet diff,% 2nd fs in total bet diff,favorite no. of matches,2nd favorite no. of matches,favorite no. of matches payin,2nd favorite no. of matches payin,favorite no. of matches diff,2nd favorite no. of matches diff,favourite hour,favourite hour payin,week no. of leagues,week no. of sports,week no. of prematch mathces,week no. of live mathces,week no. of favourite mathces,average week league risk value,week live payin,week live diff,week live margin %,week prematch payin,week prematch diff,week prematch margin %,favorite sport_Fudbal,favorite sport_Košarka,favorite sport_Ostalo,favorite sport_Stoni Tenis,favorite sport_Tenis,2nd favorite sport_Fudbal,2nd favorite sport_Hokej,2nd favorite sport_Košarka,2nd favorite sport_Nema,2nd favorite sport_Ostalo,2nd favorite sport_Stoni Tenis,2nd favorite sport_Tenis
0,2786,10,21,0.0,3200.0,0.0,3200.0,1.0,10,10130.0,6930.0,3200.0,1.0,0.0,0.0,6.933810,0.0,-36.204381,482.38,482.380952,12,6,700.0,9430.0,3.00,-0.89,33.33,449.05,0,0,1,4.223371,2.918667,0.0,-1.524861,1.254861,1.258194,8629.0,200.0,240.0,99.0,0.97,0.02,0.97,0.02,2500.0,661.0,0.0,-147.0,0.28,0.07,0.28,0.07,3591.0,2508.0,211.0,384.0,0.40,0.28,1.0,4.0,3329.0,2748.0,2886.0,-738.0,14.0,3700.0,2256,35,3081,22362,2051,0.592416,1.087537e+09,1.053392e+08,0.096860,4.935514e+08,4.102456e+07,0.083121,1,0,0,0,0,0,0,1,0,0,0,0
1,2786,11,13,0.0,2640.0,0.0,2640.0,1.0,8,3668.0,1028.0,2640.0,1.0,0.0,0.0,6.036923,0.0,-46.209538,282.15,282.153846,6,5,450.0,3218.0,-0.35,8.80,34.62,247.54,0,0,1,6.142672,3.201667,0.0,-1.500727,1.246727,1.254000,5493.0,0.0,1402.0,0.0,1.00,0.00,1.00,0.00,588.0,513.0,588.0,513.0,0.11,0.09,0.11,0.09,1718.0,1213.0,1278.0,663.0,0.31,0.22,1.0,2.0,2668.0,1025.0,615.0,1025.0,13.0,890.0,2389,35,3132,23361,2563,0.599306,1.111456e+09,6.044331e+07,0.054382,4.740170e+08,1.007177e+08,0.212477,1,0,0,0,0,0,0,0,1,0,0,0
2,2786,12,15,0.0,3000.0,0.0,3000.0,1.0,9,6895.0,3895.0,3000.0,1.0,0.0,0.0,7.251333,0.0,-23.943733,459.67,459.666667,7,5,1435.0,5460.0,1.00,4.05,95.67,364.00,0,0,1,9.694937,12.810000,0.0,-1.478235,1.282353,1.193235,7494.0,0.0,2415.0,0.0,1.00,0.00,1.00,0.00,2624.0,1582.0,1249.0,1438.0,0.35,0.21,0.35,0.21,2060.0,1984.0,-1347.0,1984.0,0.27,0.26,1.0,2.0,4939.0,1255.0,2449.0,-1549.0,17.0,1635.0,2294,34,2642,25002,2106,0.635647,8.388408e+08,5.859334e+07,0.069850,3.365150e+08,3.818964e+07,0.113486,1,0,0,0,0,0,0,0,1,0,0,0
3,2786,13,9,0.0,1000.0,0.0,1000.0,1.0,3,4081.0,3081.0,1000.0,1.0,0.0,0.0,7.330000,0.0,-45.582444,453.44,453.444444,5,2,1984.0,2097.0,1.00,-1.99,220.44,233.00,0,0,1,6.102032,3.813333,0.0,-1.982955,1.342727,1.640227,1014.0,500.0,1.0,-133.0,0.58,0.28,0.58,0.28,500.0,229.0,-133.0,-26.0,0.28,0.13,0.28,0.13,500.0,400.0,-133.0,-199.0,

In [37]:
data.dtypes

korisnik                                                           int64
week in year                                                       int64
number of tickets                                                  int64
verified                                                         float64
week deposit in                                                  float64
week deposit out                                                 float64
week deposit diff                                                float64
week deposit diff %                                              float64
frequency                                                          int64
total week payin                                                 float64
total week win                                                   float64
total week diff                                                  float64
average number of combinations                                   float64
cashout win                                        

In [38]:
data.head()

,korisnik,week in year,number of tickets,verified,week deposit in,week deposit out,week deposit diff,week deposit diff %,frequency,total week payin,total week win,total week diff,average number of combinations,cashout win,cashback win,median_odds,diff between payin and win time,median risk value on tickets,average payin on ticket,median payin on ticket,number of distinct payin on ticket value,number of different ip adress,payin on live,payin on prematch,margin on live,margin on prematch,average payin on live,average payin on prematch,number of authorization ticket,number of approved ticket,problematic_ticket_cluster,stedev odds on ticket,average hit odd,average diff between match kickoff time and ticket payin time,average diff between gray and real odds,average win odd value,average loss odd value,favorite sport payin,2nd favorite sport payin,favorite sport diff,2nd favorite sport diff,% fs in total sport diff,% 2nd fs in total sport diff,% fs in total sport payin,% 2nd fs in total sport payin,favorite league payin,2nd favorite league payin,favorite league diff,2nd favorite league diff,% fs in total league diff,% 2nd fs in total league diff,% fs in total league payin,% 2nd fs in total league payin,favorite bet payin,2nd favorite bet payin,favorite bet diff,2nd favorite bet diff,% fs in total bet diff,% 2nd fs in total bet diff,favorite no. of matches,2nd favorite no. of matches,favorite no. of matches payin,2nd favorite no. of matches payin,favorite no. of matches diff,2nd favorite no. of matches diff,favourite hour,favourite hour payin,week no. of leagues,week no. of sports,week no. of prematch mathces,week no. of live mathces,week no. of favourite mathces,average week league risk value,week live payin,week live diff,week live margin %,week prematch payin,week prematch diff,week prematch margin %,favorite sport_Fudbal,favorite sport_Košarka,favorite sport_Ostalo,favorite sport_Stoni Tenis,favorite sport_Tenis,2nd favorite sport_Fudbal,2nd favorite sport_Hokej,2nd favorite sport_Košarka,2nd favorite sport_Nema,2nd favorite sport_Ostalo,2nd favorite sport_Stoni Tenis,2nd favorite sport_Tenis
0,2786,10,21,0.0,3200.0,0.0,3200.0,1.0,10,10130.0,6930.0,3200.0,1.0,0.0,0.0,6.933810,0.0,-36.204381,482.38,482.380952,12,6,700.0,9430.0,3.00,-0.89,33.33,449.05,0,0,1,4.223371,2.918667,0.0,-1.524861,1.254861,1.258194,8629.0,200.0,240.0,99.0,0.97,0.02,0.97,0.02,2500.0,661.0,0.0,-147.0,0.28,0.07,0.28,0.07,3591.0,2508.0,211.0,384.0,0.40,0.28,1.0,4.0,3329.0,2748.0,2886.0,-738.0,14.0,3700.0,2256,35,3081,22362,2051,0.592416,1.087537e+09,1.053392e+08,0.096860,4.935514e+08,4.102456e+07,0.083121,1,0,0,0,0,0,0,1,0,0,0,0
1,2786,11,13,0.0,2640.0,0.0,2640.0,1.0,8,3668.0,1028.0,2640.0,1.0,0.0,0.0,6.036923,0.0,-46.209538,282.15,282.153846,6,5,450.0,3218.0,-0.35,8.80,34.62,247.54,0,0,1,6.142672,3.201667,0.0,-1.500727,1.246727,1.254000,5493.0,0.0,1402.0,0.0,1.00,0.00,1.00,0.00,588.0,513.0,588.0,513.0,0.11,0.09,0.11,0.09,1718.0,1213.0,1278.0,663.0,0.31,0.22,1.0,2.0,2668.0,1025.0,615.0,1025.0,13.0,890.0,2389,35,3132,23361,2563,0.599306,1.111456e+09,6.044331e+07,0.054382,4.740170e+08,1.007177e+08,0.212477,1,0,0,0,0,0,0,0,1,0,0,0
2,2786,12,15,0.0,3000.0,0.0,3000.0,1.0,9,6895.0,3895.0,3000.0,1.0,0.0,0.0,7.251333,0.0,-23.943733,459.67,459.666667,7,5,1435.0,5460.0,1.00,4.05,95.67,364.00,0,0,1,9.694937,12.810000,0.0,-1.478235,1.282353,1.193235,7494.0,0.0,2415.0,0.0,1.00,0.00,1.00,0.00,2624.0,1582.0,1249.0,1438.0,0.35,0.21,0.35,0.21,2060.0,1984.0,-1347.0,1984.0,0.27,0.26,1.0,2.0,4939.0,1255.0,2449.0,-1549.0,17.0,1635.0,2294,34,2642,25002,2106,0.635647,8.388408e+08,5.859334e+07,0.069850,3.365150e+08,3.818964e+07,0.113486,1,0,0,0,0,0,0,0,1,0,0,0
3,2786,13,9,0.0,1000.0,0.0,1000.0,1.0,3,4081.0,3081.0,1000.0,1.0,0.0,0.0,7.330000,0.0,-45.582444,453.44,453.444444,5,2,1984.0,2097.0,1.00,-1.99,220.44,233.00,0,0,1,6.102032,3.813333,0.0,-1.982955,1.342727,1.640227,1014.0,500.0,1.0,-133.0,0.58,0.28,0.58,0.28,500.0,229.0,-133.0,-26.0,0.28,0.13,0.28,0.13,500.0,400.0,-133.0,-199.0,

In [39]:
data.shape

(218321, 91)

### PODACI SU SREDJENI ZA DALJI RAD

# 2. KREIRANJE FUNKCIJA ZA NEURONSKU MREZU

## 2.1 KREIRANJE FUNKCIJA ZA STANDARDIZACIJU I ANALIZU GLAVNIH KOMPONENTI

In [40]:
def standardizacijaIDoradaPodataka(podaci:pd.DataFrame):
    from sklearn.preprocessing import StandardScaler
    
    korKolona = podaci['korisnik']
    korNedelja = podaci['week in year']
    
    skaler = StandardScaler()
    stdpodaci = skaler.fit_transform(podaci.drop(columns=['korisnik','week in year']))
    kolone = podaci.drop(columns=['korisnik','week in year']).columns.to_list()
    stdpodaci = pd.DataFrame(stdpodaci)
    
    stdpodaci.columns = kolone
    stdpodaci['korisnik'] = korKolona
    stdpodaci['week in year'] = korNedelja
    
    stdpodaci = stdpodaci[stdpodaci.columns[-2:].tolist() + stdpodaci.columns[:-2].tolist()]
    
    return stdpodaci

In [41]:
def analizaGlavnihKomponenti(podaci:pd.DataFrame, brojGK):
    from sklearn.decomposition import PCA
    from matplotlib import pyplot as plt
    
    korKolona = podaci['korisnik']
    korNedelja = podaci['week in year']
    totWeekDif = podaci['total week diff']
    
    ostaliPodaci = podaci.drop(columns=['korisnik', 'week in year', 'total week diff'])
    
    pca = PCA(n_components = brojGK)
    rezPca = pca.fit_transform(ostaliPodaci)
    objasnjenaVarijansa = pca.explained_variance_ratio_
    print(f"Procenat objasnjenog varijabliteta pomocu glavnih komponenti iznosi: {sum(objasnjenaVarijansa)}")
    indeksi = [i for i in range(1,brojGK+1)]
    plt.plot(indeksi, objasnjenaVarijansa)
    plt.xticks(np.arange(start=1, stop=brojGK+1, step=1))
    plt.title("PCA - objasnjeni varijabilitet")
    plt.ylabel("Procenat objasnjenog varijabiliteta")
    plt.xlabel("Redni broj komponente")
    plt.grid(True)
    plt.show()
    gkDf = pd.DataFrame(pca.fit_transform(rezPca))
    gkDf.columns = [f"Glavna komp. {i}" for i in range(1, brojGK+1)]
    
    gkDf['total week diff'] = totWeekDif
    gkDf['korisnik'] = korKolona
    gkDf['week in year'] = korNedelja
    
    gkDf = gkDf[gkDf.columns[-2:].tolist() + gkDf.columns[:-2].tolist()]
    gkDf = standardizacijaIDoradaPodataka(gkDf)
    
    return gkDf

## 2.2 KREIRANJE FUNKCIJA NEURONSKE MREZE

In [42]:
def neuronskaMreza(df:pd.DataFrame, brUlaza, rbNedelje, idKorisnika = None):
    import numpy as np
    from keras.models import Sequential
    from keras.layers import Dense
    from sklearn.model_selection import train_test_split 
    from sklearn.metrics import mean_squared_error, explained_variance_score
    
    trdf = df
    testdf = df.copy()
    
    testdf = testdf.loc[testdf['week in year'] == rbNedelje]
    korKolona = testdf['korisnik']
    testdf.drop(columns=["korisnik", "week in year"], inplace=True)
    xtest = testdf.drop(columns=["total week diff"])
    ytest = testdf["total week diff"]
    
    trdf = trdf.loc[trdf['week in year'] < rbNedelje]
    trdf.drop(columns=["korisnik", "week in year"], inplace=True)
    xtr = trdf.drop(columns=["total week diff"])
    ytr = trdf["total week diff"]
    
    model = Sequential()
    model.add(Dense(1, input_dim=brUlaza))
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    model.fit(xtr, ytr, epochs=5, batch_size=32, verbose=0)
    
    ypred = model.predict(xtest)
    
    dfResenje = pd.DataFrame(ytest)
    dfResenje.columns = ["Stvarna vrednost"]
    dfResenje["Predvidjena vrednost"] = ypred
    
    dfResenje['Korisnik'] = korKolona
    dfResenje = dfResenje[dfResenje.columns[-1:].tolist() + dfResenje.columns[:-1].tolist()]
    
    if idKorisnika != None:
        stv = dfResenje.loc[dfResenje['Korisnik'] == idKorisnika, 'Stvarna vrednost']
        pred = dfResenje.loc[dfResenje['Korisnik'] == idKorisnika, 'Predvidjena vrednost']
        print(f"\nZa korisnika ciji je ID {idKorisnika}, stvarna vrednost je {stv.values}, a neuronska mreza je predvidela {pred.values}\n")
    
    
    print(f"Uzorak 10 korisnika i njihovih stvarnih i predvidjenih vrednosti u {rbNedelje}. nedelji 2022. godine:\n")
    print(dfResenje.sample(10))
    
    return dfResenje

## 2.3 KREIRANJE IZVRSNE FUNKCIJE ZA KORISNIKA

In [43]:
def izvrsnaFunkcija(data:pd.DataFrame, brNedelje, idKorisnika = None, brGlKomp = None):
    
    stdData = standardizacijaIDoradaPodataka(data)
    
    if brGlKomp is None:
        brGlKomp = 88
    else:
        stdData = analizaGlavnihKomponenti(stdData, brGlKomp)
        
    rezDf = neuronskaMreza(stdData, brGlKomp, brNedelje, idKorisnika)
    
    return rezDf

# 3. TEMPLEJT ZA RAD

In [45]:
# u izvrsnu funkciju se redom unose sledeci argumenti:
# - datafrejm sa odgovarajucim atributima za dalju analizu
# - redni broj nedelje u godini za koju se vrsi predikcija (vrednost izmedju 10 i 50)
# - ID korisnika za kog zelimo da vidimo stvarnu i predvidjenu vrednost (opciono)
# - broj glavnih komponenti koje zelimo da koristi neuronska mreza (opciono, ukoliko se ne unese nm radi sa svim atributima)

#konacniDf = izvrsnaFunkcija(data, brNedelje, idKorisnika=id, brGlKomp=brGk)